## Cleaner pipeline

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
data_raw = pd.read_excel("Логи.xlsx")
data_raw.head()

BadZipFile: ignored

In [ ]:
ERROR_STEMS = [
        # internal
        "Данные не могут быть обработаны по причине",
        "Произошла ошибка импорта",
        "Данные не могут быть проимпортированы по причине",

        # common
        "Connection ID",
        "Execution Timeout Expired",
        "Unable to parse request",
        "Unable to process",
        "Unable to send",
        "Unhandled error",
        "Для команды интеграции",
        "Ошибка обработки события",
        "Ошибка при обращении к сервису ЕИС",
        "Проверка приглашений",
        "Событие обработано с ошибками",
        "Exception while Executing",


        # Database
        "An exception occurred in the database while saving changes",
        "An exception occurred while iterating over the results of a query",
        "Could not execute query",
        "Could not execute command",
        "Execution error",
        "Failed executing DbCommand",
        "Failed to execute query batch",
        "NHibernate.Exceptions.GenericADOException",
        "The database operation was expected",


        # Operations
        "OpComplete",
        "System.InvalidOperationException",

        # RabbitMQ
        "R-FAULT rabbitmq",
        "S-FAULT rabbitmq",
        "Ошибка при обработке запроса",
        "Ошибка при обработке события",
        "Ошибка публикации события",

        # МЧД ошибки
        "МЧД",
        "Необработанная ошибка валидации МЧД"
    ]

In [ ]:
ERROR_STEMS

In [ ]:
def print_whole_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [ ]:
import re

def replace_parameters(x: str):
    return re.sub(r'[\:\=]\s?[^\,\]\s]+', '', x)

def apply_stemming(x: str):
    for stem in ERROR_STEMS:
        if x.startswith(stem):
            return stem
    return x

In [ ]:
data_raw['log_short'] = data_raw['log'].apply(replace_parameters).apply(apply_stemming)

In [ ]:
len(data_raw['log_short'].unique())

In [ ]:
print_whole_df(data_raw['log_short'].value_counts())

In [ ]:
data_raw['log_short'].value_counts().to_csv("reduced_stemmed_paramed_logs.csv", encoding='utf-8-sig')

In [ ]:
data_raw.head()

In [ ]:
data_raw.to_csv("logs_with_short.csv", encoding="utf-8-sig")

## Lookup Table rework

In [ ]:
look_up_table_old = pd.read_excel("ErrorLookUpTable.xlsx")
look_up_table_old.head()

,Коммуникации с внешними системами,Ошибка внутренних сервисов,Ошибки БД,Неизвестная ошибка,Не ошибка,Ошибка валидации
0,Данные не могут быть проимпортированы по причине,Execution error,Could not execute query,Unexpected error occured,Immediate job execution failed,МЧД
1,Unable to get integration token,Ошибка обработки запроса на регистрацию/измене...,Failed executing DbCommand,Exception SendCommand,Response already has started,Необработанная ошибка валидации МЧД
2,Unable to send,Execution Timeout Expired,An exception occurred while iterating over the...,Ошибка выполнения подписчика на событие заверш...,Ошибка при расчёте достижений компании.,NaN
3,Ошибка получения файлов из каталога nsi ЕИС,The wait operation timed out,Failed to execute query batch,job error,NaN,NaN
4,S-FAULT rabbitmq,Pp3.Contracts.Dto.Contract.Action.ContractActi...,Unable to get data,Unhandled error,NaN,NaN


In [ ]:
logs_classes = look_up_table_old.columns
index2class = {i: v for (i, v) in enumerate(logs_classes)}
class2index = {v: k for (k, v) in index2class.items()}

In [ ]:
index2class, class2index

({0: 'Коммуникации с внешними системами',
  1: 'Ошибка внутренних сервисов',
  2: 'Ошибки БД',
  3: 'Неизвестная ошибка',
  4: 'Не ошибка',
  5: 'Ошибка валидации'},
 {'Коммуникации с внешними системами': 0,
  'Ошибка внутренних сервисов': 1,
  'Ошибки БД': 2,
  'Неизвестная ошибка': 3,
  'Не ошибка': 4,
  'Ошибка валидации': 5})

In [ ]:
logtype2class_index = {logtype: class2index[log_class] for log_class in logs_classes for logtype in look_up_table_old[log_class] if logtype is not np.nan}
logtype2class_index

{'Данные не могут быть проимпортированы по причине': 0,
 'Unable to get integration token': 0,
 'Unable to send': 0,
 'Ошибка получения файлов из каталога nsi ЕИС': 0,
 'S-FAULT rabbitmq': 0,
 'Ошибка при обработке события': 0,
 'Невозможно запросить данные из УАС': 0,
 'Ошибка запроса данных из УАС': 0,
 'R-FAULT rabbitmq': 0,
 'Невозможно скачать файл по ссылке': 0,
 'Ошибка авторизации в интеграционном апи': 0,
 'Не найден КПГЗ': 0,
 'Unable to upload files': 0,
 'Ошибка запроса статуса самозанятого': 0,
 'Произошла ошибка импорта': 0,
 'Для команды интеграции': 0,
 'Unable to parse request': 0,
 'Архив (УАС) не загрузился/не загружался': 0,
 'NHibernate.Exceptions.GenericADOException': 0,
 'Kaluga workflowId, Сессия умерла': 0,
 'Ошибка обращения к api внешней системы ЕГРЮЛ ДИТ': 0,
 'Ошибка при вызове сервиса ЕИС': 0,
 'Ошибка при запросе списка интеграционного обмена.': 0,
 'Не удалось скачать файл из ЕИС по ссылке': 0,
 'Unable to set failed for command, type': 0,
 'Ошибка досту

In [ ]:
logtype2class_name = {k: index2class[v] for (k, v) in logtype2class_index.items()}
logtype2class_name

{'Данные не могут быть проимпортированы по причине': 'Коммуникации с внешними системами',
 'Unable to get integration token': 'Коммуникации с внешними системами',
 'Unable to send': 'Коммуникации с внешними системами',
 'Ошибка получения файлов из каталога nsi ЕИС': 'Коммуникации с внешними системами',
 'S-FAULT rabbitmq': 'Коммуникации с внешними системами',
 'Ошибка при обработке события': 'Коммуникации с внешними системами',
 'Невозможно запросить данные из УАС': 'Коммуникации с внешними системами',
 'Ошибка запроса данных из УАС': 'Коммуникации с внешними системами',
 'R-FAULT rabbitmq': 'Коммуникации с внешними системами',
 'Невозможно скачать файл по ссылке': 'Коммуникации с внешними системами',
 'Ошибка авторизации в интеграционном апи': 'Коммуникации с внешними системами',
 'Не найден КПГЗ': 'Коммуникации с внешними системами',
 'Unable to upload files': 'Коммуникации с внешними системами',
 'Ошибка запроса статуса самозанятого': 'Коммуникации с внешними системами',
 'Произошла

### Dump jsons

In [ ]:
import json

In [ ]:
def write_json(obj, fname):
    with open(fname, 'w') as f:
        s = json.dumps(obj, indent=4, ensure_ascii=False)
        f.write(s)

In [ ]:
write_json(index2class, "class_index2class_name.json")
write_json(logtype2class_index, "log_type2class_index.json")

## Classifier

In [ ]:
import json
import re
from dataclasses import dataclass


@dataclass
class LogInfo:
    log_type: str
    cat_name: str


class LogClassifier:
    def __init__(self):
        self.index2class = json.loads(open("class_index2class_name.json", 'r').read())
        self.logtype2class_index = json.loads(open("log_type2class_index.json", 'r').read())
        self.unk = 'Unhandled error'

    def replace_parameters(self, x: str):
        return re.sub(r'[\:\=]\s?[^\,\]\s]+', '', x)

    def apply_stemming(self, x: str):
        for stem in self.logtype2class_index.keys():
            if x.startswith(stem):
                return stem
        return self.unk

    def process_raw(self, log_s: str):
        return self.apply_stemming(self.replace_parameters(log_s))

    def predict(self, log_s: str):
        log_type = self.process_raw(log_s)
        cat_id = self.logtype2class_index[log_type]
        cat_name = self.index2class[str(cat_id)]

        return LogInfo(log_type, cat_name)

In [ ]:
cls = LogClassifier()

In [ ]:
cls.predict('The wait operation timed out')

LogInfo(log_type='The wait operation timed out', cat_name='Ошибка внутренних сервисов')

## Torture data a little bit more

In [ ]:
import numpy as np
import pandas as pd
import nltk
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

In [ ]:
def replace_parameters(x: str):
    return re.sub(r'[\:\=]\s?[^\,\]\s]+', '', x)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data = pd.read_excel("Логи.xlsx")

In [ ]:
data.head()

,id,create_date,log
0,6424198940e01f33f8b596fa9fe7391c,2023-10-09 00:00:01,Unable to get integration token
1,9851574021c269d401accc3a1381259a,2023-10-09 00:00:02,Unable to get integration token
2,35f4f5fc4fdd12717347c2064729930e,2023-10-09 00:00:02,Unable to get integration token
3,49f9dc2cb58ab8db9f59b2487fbe0768,2023-10-09 00:00:02,Unable to get integration token
4,4f4830735dfc264fb40d1424e3e02bb2,2023-10-09 00:00:08,Pp3.Contracts.Dto.Contract.Action.ContractActi...


In [ ]:
data['create_date'] = pd.to_datetime(data['create_date'])

In [ ]:
data['logs_without_params'] = data['log'].apply(replace_parameters)

In [ ]:
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

sw = set(stopwords.words('english') + stopwords.words('russian'))

def tokenize(sent):
    return wordpunct_tokenize(sent)

def ishex(s):
    return not set(s) - set("abcdef0123456789")

def remove_words(sent):
    return [w for w in sent if w not in sw and not ishex(w) and any(c.isalpha() for c in w)]

def process_corpus(corpus):
    tokenized = [tokenize(sent) for sent in tqdm(corpus, desc="Tokenization")]
    clean_corpus = [remove_words(sent) for sent in tqdm(tokenized, desc="Removing words")]
    return clean_corpus

corpus = data['logs_without_params'].str.lower().to_list()
clean_corpus = process_corpus(corpus)

Tokenization:   0%|          | 0/88855 [00:00<?, ?it/s]

Removing stopwords:  68%|██████▊   | 60154/88855 [25:22<12:06, 39.50it/s]   


Removing words:   0%|          | 0/88855 [00:00<?, ?it/s]

In [ ]:
from collections import Counter

counter = Counter()

for sent in tqdm(clean_corpus, desc="Creating vocab"):
    counter.update(sent)

Creating vocab:   0%|          | 0/88855 [00:00<?, ?it/s]

In [ ]:
def build_vocab(counter: Counter, min_freq=3):
    most_common = counter.most_common()
    vocab = set([word for (word, freq) in most_common if freq >= min_freq])
    return vocab

In [ ]:
counter.most_common()

[('данные', 46145),
 ('причине', 46116),
 ('могут', 46067),
 ('packagedatetime', 45978),
 ('packageguid', 45976),
 ('сущности', 45962),
 ('импортирована', 45961),
 ('поздняя', 45961),
 ('версия', 45961),
 ('проимпортированы', 44392),
 ('this_', 34124),
 ('id', 18828),
 ('offer0_', 13970),
 ('join', 13439),
 ('reg2_', 12867),
 ('execution', 12752),
 ('inner', 12062),
 ('operation', 10264),
 ('timeout', 10175),
 ('error', 8684),
 ('this_0_', 8611),
 ('type', 8433),
 ('job', 8396),
 ('deleted', 7638),
 ('ошибка', 7354),
 ('contract', 7349),
 ('key', 6787),
 ('args', 6783),
 ('y0_', 6618),
 ('запроса', 6471),
 ('обработки', 6469),
 ('компании', 6458),
 ('данных', 6452),
 ('изменение', 6431),
 ('регистрацию', 6403),
 ('unable', 5669),
 ('query', 5642),
 ('execute', 5450),
 ('could', 5374),
 ('cast', 5099),
 ('completion', 5086),
 ('expired', 5072),
 ('period', 5072),
 ('elapsed', 5072),
 ('prior', 5072),
 ('server', 5072),
 ('responding', 5072),
 ('wait', 5070),
 ('timed', 5070),
 ('hierarc

In [ ]:
vocab = build_vocab(counter)

In [ ]:
len(vocab)

1884

In [ ]:
def process_sent(sent, vocab=vocab):
    sent = replace_parameters(sent)
    tokenized = tokenize(sent.lower())
    cleaned = remove_words(tokenized)
    vocabed = [word for word in cleaned if word in vocab]
    return ' '.join(vocabed)

In [ ]:
data['clean_logs'] = data['log'].apply(process_sent)

In [ ]:
len(data['clean_logs'].unique())

372

In [ ]:
data['clean_logs'].value_counts()

данные могут проимпортированы причине импортирована поздняя версия сущности packageguid packagedatetime                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 